# Analysis List Moves

Let's try to do something with our list of moves ...

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

In [3]:
data = pd.read_csv('../data_othello.csv')
display(data.head())
input_shape = data.shape
print("Input shape : {}".format(input_shape))

,Player 1,PLayer 2,Liste Moves,Score,UCB 1,UCB2,it 1,it 2,th 1,th 2,depth 1,depth 2
0,Random,Random,"[((4, 5), 1), ((3, 5), -1), ((2, 6), 1), ((5, ...",-8,0.14,0.14,20.0,20.0,10.0,10.0,3.0,3.0
1,Random,Random,"[((2, 3), 1), ((2, 2), -1), ((5, 4), 1), ((5, ...",28,0.14,0.14,20.0,20.0,10.0,10.0,3.0,3.0
2,Random,Random,"[((3, 2), 1), ((4, 2), -1), ((5, 1), 1), ((2, ...",-8,0.14,0.14,20.0,20.0,10.0,10.0,3.0,3.0
3,Random,Random,"[((5, 4), 1), ((3, 5), -1), ((2, 4), 1), ((1, ...",2,0.14,0.14,20.0,20.0,10.0,10.0,3.0,3.0
4,Random,Random,"[((3, 2), 1), ((2, 4), -1), ((4, 5), 1), ((4, ...",10,0.14,0.14,20.0,20.0,10.0,10.0,3.0,3.0


Input shape : (20, 12)


This time, the only thing that matter is the liste of moves and the win (for the label) so let go for some data transformation !

In [4]:
def split_lst_move(row) :

    row['Liste Moves'] = row['Liste Moves'].replace('[', '')
    row['Liste Moves'] = row['Liste Moves'].replace(']', '')
    row['Liste Moves'] = row['Liste Moves'].replace('((', '')
    l_row = row['Liste Moves'].split("), ")
    l_row[-1] = l_row[-1][:-1]
    # print(row['win'])
    # row = row.drop(['Liste Moves'], axis = 1)
    if len(l_row) == 120 : # Drop party that ended before filled the whole board (don't know how to do for the density map otherwise)
        for i in range(0,len(l_row)-1, 2):
            if (int(l_row[i + 1]) < 0 and int(row['Score']) < 0) or (int(l_row[i + 1]) > 0 and int(row['Score']) > 0) :
                n_row = l_row[i].split(", ")
                row[str(round(i/2) ) + 'x'] = n_row[0]
                row[str(round(i/2) ) + 'y'] = n_row[1]
                # row[str(round(i/2)) + 'p'] = l_row[i + 1]
    return row[1:]

# row1 = df.iloc[1]
# print(split_lst_move(row1))

In [5]:
df = data.drop(data.iloc[:, :2].columns, axis=1)
df = df.drop(df.iloc[:,2:].columns, axis=1)
# df = df[:500] # Work on a smaller part for try
df = df.apply(lambda row: split_lst_move(row), axis = 1)
df.head()
# pd.DataFrame(df['Liste Moves'].tolist(), index=df.index)
# df.head()

,0x,0y,10x,10y,11x,11y,12x,12y,13x,13y,...,5y,6x,6y,7x,7y,8x,8y,9x,9y,Score
0,NaN,NaN,NaN,NaN,5,2,NaN,NaN,1,1,...,2,NaN,NaN,5,3,NaN,NaN,7,5,-8
1,2,3,0,3,NaN,NaN,1,1,NaN,NaN,...,NaN,5,6,NaN,NaN,2,4,NaN,NaN,28
2,NaN,NaN,NaN,NaN,4,0,NaN,NaN,6,1,...,2,NaN,NaN,2,2,NaN,NaN,4,1,-8
3,5,4,2,6,NaN,NaN,5,6,NaN,NaN,...,NaN,2,2,NaN,NaN,1,4,NaN,NaN,2
4,3,2,5,2,NaN,NaN,1,2,NaN,NaN,...,NaN,4,1,NaN,NaN,1,3,NaN,NaN,10


In [6]:
def generate_z(row, Z):
    row = row.dropna()
    for i in range(0, len(row) - 2, 2):
        Z[int(row[i])][int(row[i+1])] += 1

In [7]:
# Sample data
side = np.linspace(0,7,8)
X,Y = np.meshgrid(side,side)
Z = np.zeros((8,8))
df.apply(lambda row: generate_z(row, Z), axis = 1)

# Plot the density map using nearest-neighbor interpolation
# fig = plt.pcolormesh(X,Y,Z)
# plt.title('Density Map')
# plt.show()
# fig = fig.get_figure()
# fig.savefig('./images/Density_Map.png')

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
dtype: object

This density map was generated by keeping only the moves played by the winning player. Most of the dataset was generated using randomness and MCTS. MinMax and Alpha-Beta using a density map doesn't influence to much our result.

<img src="./images/Density_Map.png">


In [8]:
def split_lst_move2(row) :
    row['Liste Moves'] = row['Liste Moves'].replace('[', '')
    row['Liste Moves'] = row['Liste Moves'].replace(']', '')
    row['Liste Moves'] = row['Liste Moves'].replace('((', '')
    l_row = row['Liste Moves'].split("), ")
    l_row[-1] = l_row[-1][:-1]
    for i in range(0,len(l_row)-1, 2):
        row[str(l_row[i])] = int(i/2)
    return row[1:]

In [9]:
df2 = data.drop(data.iloc[:, :2].columns, axis=1)
df2 = df2.drop(df2.iloc[:,2:].columns, axis=1)
df2 = df2.apply(lambda row: split_lst_move2(row), axis = 1)
df2.head()

,"0, 0","0, 1","0, 2","0, 3","0, 4","0, 5","0, 6","0, 7","1, 0","1, 1",...,"6, 7","7, 0","7, 1","7, 2","7, 3","7, 4","7, 5","7, 6","7, 7",Score
0,33,14,28,45,54,53,44.0,55,49,13,...,39,59,34,15,30,50,9,36,38,-8
1,19,30,25,10,55,38,27.0,34,56,12,...,7,52,51,48,31,24,33,44,28,28
2,59,55,23,42,41,52,57.0,33,32,22,...,50,21,24,15,18,51,43,56,39,-8
3,52,37,39,44,18,35,14.0,50,41,53,...,29,58,57,51,40,20,28,34,49,2
4,32,53,45,52,41,58,59.0,47,57,15,...,56,42,48,36,29,44,54,51,40,10


In [10]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

2022-12-02 16:49:40.470943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 16:49:41.111137: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 16:49:41.203947: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 16:49:41.203977: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

The idea is to see if it is possible to predict who wins with all the moves played in order. For the moment I test without indicating who has played the moves but just the order. 

In [25]:
df_test = df2.copy()

df_train = df_test.sample(frac = 0.7, random_state = 0)
df_valid = df_test.drop(df_train.index)

X_train = df_train.drop('Score', axis = 1)
X_valid = df_valid.drop('Score', axis = 1)
y_train = df_train['Score']
y_test = df_test['Score']

input_shape = [X_train.shape[1]]

In [28]:
modek = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape = input_shape),
    layers.Dropout(0.3),
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1)
])

In [27]:
model.compile(
    optimizer='adam',
    loss='mae'
)